In [17]:
import pandas as pd
import math
import numpy as np
from shapely.geometry import Point, Polygon
import copy
import time
import re
from datetime import date, datetime, timedelta
from shapely.geometry import Point, Polygon
import copy
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

<h1>Replica of dataset generator template with tests<h1>

In [18]:
neighbourhood_data_name = "SF_neighborhoods.csv"
sfpd_data_name = "incidents_new_data.csv" #current range 12/11/2020 to 28/01/2021
business_data_name = "business_new_data.csv"
final_data_name = "new_final_data.csv"
start_day = 18
start_month = 11
start_year = 2020
end_day = 28
end_month = 1
end_year = 2021

In [19]:
# generate all required date values, no adjustment required from this point onwards
start_date_string = str(start_year)+'-'+str(start_month)+'-'+str(start_day)
start_date = date(start_year,start_month,start_day)
one_year_before_start_date = start_date - timedelta(days=365)
end_date = date (end_year,end_month,end_day)
date_after_end_date = end_date + timedelta(days=1)
future_year_increment = 50
future_date = str(end_year+future_year_increment)+'-'+str(end_month)+'-'+str(end_day)
future_year = end_year + future_year_increment
business_start_date = str(start_month)+'-'+str(start_day)+'-'+str(start_year)
business_end_date = str(end_month)+'-'+str(end_day)+'-'+str(end_year)

<h1> Get neighbourhood data <h1>

In [7]:
# open neighbourhood data file
def open_neighborhoods():
    df = pd.read_csv(neighbourhood_data_name)
    return df

In [8]:
# get list of neighbourhoods from neighbourhood datafile
def get_neighborhoods(df):
    neighborhoods = []
    num_neighborhoods = len(df.index)
    for x in range(0,num_neighborhoods):
        neighborhoods.append(df.iloc[x,2])
    return neighborhoods

In [9]:
# generate list of polygons representing each neighbourhood
def generate_neighborhood_polygons(df):
    neighborhoods = []
    num_neighborhoods = len(df.index)
    for x in range(0,num_neighborhoods):
        raw_polygon_data = df.iloc[x,1]
        neighborhood_polygon = generate_polygon(raw_polygon_data)
        neighborhood_name = df.iloc[x,2]
        neighborhood_tuple = (neighborhood_polygon,neighborhood_name)
        neighborhoods.append(neighborhood_tuple)
    return neighborhoods

In [10]:
# generate a polygon
def generate_polygon(raw_polygon_data):
    raw_polygon_data = raw_polygon_data[16:-3]
    coords = ''
    coords_list = []
    for char in raw_polygon_data:
        if char == ' ' or char == ',':
            if coords != '':
                coords_list.append(float(coords))
            coords = ''
        else:
            coords = coords+char
    coords_list.append(float(coords))
    x = 0
    tuple_list = []
    while x < len(coords_list):
        next_tuple = (coords_list[x+1],coords_list[x])
        tuple_list.append(next_tuple)
        x+=2
    polygon = Polygon(tuple_list)
    return polygon

In [11]:
# open neighbourhoods file and obtain list of neighbourhoods and polygons representing their shape
df = open_neighborhoods()
neighborhoods = get_neighborhoods(df)
num_neighbourhoods = len(neighborhoods)
neighborhood_polygons = generate_neighborhood_polygons(df)

<h1> Get incident report data <h1>

In [12]:
# open sfpd data file
def open_sfpd_data():
    df = pd.read_csv(sfpd_data_name)
    return df

In [13]:
#identify total number of records and remove uncategorised crimes
def remove_uncategorised_incidents(df):
    #identify total number of records and remove uncategorised crimes
    total_records = len(df.index)
    categorised_crimes = df['Incident Category'].count()
    uncategorised_crimes = total_records - categorised_crimes
    if uncategorised_crimes != 0:
        #drop blank incident categories and recheck values
        df = df.dropna(subset=['Incident Category'])
        df = remove_uncategorised_incidents(df)
    return df

In [14]:
# drop rows with missing values
def remove_rows_with_missing_values(df):
    df = df.dropna()
    return df

In [15]:
# drop columns deemed uneccessary
def drop_unnecessary_incident_columns(df):
    df = df.drop(columns=['Report Datetime','Row ID', 'Incident ID',
                          'Incident Number','CAD Number','Report Type Code',
                          'Report Type Description', 'Filed Online',
                          'Incident Code','Incident Subcategory',
                          'Incident Description', 'Resolution', 'Intersection',
                          'CNN','Analysis Neighborhood',
                          'Supervisor District', 'point', 'SF Find Neighborhoods',
                          'Current Police Districts', 'Current Supervisor Districts',
                          'Analysis Neighborhoods','HSOC Zones as of 2018-06-05',
                          'OWED Public Spaces',
                          'Central Market/Tenderloin Boundary Polygon - Updated',
                          'Parks Alliance CPSI (27+TL sites)','ESNCAG - Boundary File',
                          'Areas of Vulnerability, 2016'])
    return df

In [16]:
# remove rows deemed as non-crime
def remove_rows_of_non_crime(df):
    df = df.loc[~df['Incident Category'].str.contains('Case Closure')]
    df = df.loc[~df['Incident Category'].str.contains('Courtesy Report')]
    df = df.loc[~df['Incident Category'].str.contains('Lost Property')]
    df = df.loc[~df['Incident Category'].str.contains('Missing Person')]
    df = df.loc[~df['Incident Category'].str.contains('Non-Criminal')]
    df = df.loc[~df['Incident Category'].str.contains('Suicide')]
    df = df.loc[~df['Incident Category'].str.contains('Vehicle Misplaced')]
    df = df.loc[~df['Incident Category'].str.contains('Warrant')]
    return df

In [17]:
# remove all rows prior to pre-calculated date
def remove_crime_prior_to_start_date(df):
    df['Incident Date'] = pd.to_datetime(df['Incident Date'],dayfirst=True)
    df = df[~(df['Incident Date'].dt.date < one_year_before_start_date)]
    df = df[~(df['Incident Date'].dt.date > end_date)]
    return df

In [18]:
# convert gps coordinates of each row into a neighbourhood by placing each point inside a neighbourhood polygon
def convert_gps_to_neighborhood(df,neighborhood_polygons):
    df['Neighborhood'] = np.nan
    num_records = len(df.index)
    for incident_num in range(0,num_records):
        df.iloc[incident_num,8] = ''
        shortest_distance = 999999999
        closest_neighborhood = ''
        latitude = float(df.iloc[incident_num,6])
        longitude = float(df.iloc[incident_num,7])
        point = Point(latitude,longitude)
        for poly_tuple in neighborhood_polygons:
            if poly_tuple[0].contains(point) or poly_tuple[0].touches(point):
                df.iloc[incident_num,8] = poly_tuple[1]
                break
            else:
                distance_to_neighborhood = point.distance(poly_tuple[0])
                if distance_to_neighborhood < shortest_distance:
                    shortest_distance = distance_to_neighborhood
                    closest_neighborhood = poly_tuple[1]
        if df.iloc[incident_num,8] == '':
            #place incident in closest neighborhood
            df.iloc[incident_num,8] = closest_neighborhood
            #shortest distance is over 1 mile away
            if shortest_distance > 0.018:
                print('WARNING!!! GPS coordinates for row',incident_num,'do not appear to be within 1 mile of a San Francisco Neighbourhood')
    return df

In [19]:
# drop gps coordinates after determining neighbourhood
def remove_gps_coordinates(df):
    df = df.drop(columns=['Latitude','Longitude'])
    return df

In [20]:
# group incidents per day per neighbourhood
def get_reports_per_day_per_neighborhood(df):
    df['Todays Reports'] = 1
    df['Incident Date'] = pd.to_datetime(df['Incident Date'],dayfirst=True)
    df = df.groupby(['Incident Date',
                     'Incident Year', 
                     'Neighborhood',
                     'Incident Day of Week'
                    ]).count()
    df = df.drop(columns=[ 'Incident Datetime','Incident Time',
                           'Incident Category'])
    df = df.reset_index()
    return df

In [21]:
# insert rows for any neighbourhood with zero crime on any given day
def check_for_missing_neighborhoods(df,neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(df.index)
    current_date = df.iloc[0,0]
    zero_crime_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        if df.iloc[record,0] == current_date:
            zero_crime_neighborhoods.remove(df.iloc[record,2])
        if record != num_records-1:
            if df.iloc[record+1,0] != current_date:
                for neighborhood in zero_crime_neighborhoods:
                    new_row = {'Incident Date' : current_date,
                               'Incident Year' : df.iloc[record-1,1],
                               'Neighborhood' : neighborhood,
                               'Incident Day of Week' : current_date.strftime('%A'),
                               'Todays Reports' : 0}
                    new_rows = new_rows.append(new_row,ignore_index=True)
                zero_crime_neighborhoods = copy.deepcopy(neighborhoods)
                if df.iloc[record,0] == current_date + timedelta(days=1):
                    zero_crime_neighborhoods.remove(df.iloc[record,2])
                current_date = current_date + timedelta(days=1)
    for neighborhood in zero_crime_neighborhoods:
        new_row = {'Incident Date' : current_date,
                    'Incident Year' : df.iloc[len(df.index)-1,1],
                    'Neighborhood' : neighborhood,
                    'Incident Day of Week' : current_date.strftime('%A'),
                    'Todays Reports' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [df,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Incident Date','Neighborhood'])
    return new_record

In [22]:
# generate dataframe of last 14 days incidents
def get_last_14_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days reports']
    return transitional_data

In [23]:
# generate dataframe of last 28 days incidents
def get_last_28_days(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days reports']
    return transitional_data

In [24]:
# generate dataframe of historic incidents
def generate_new_date_data(df):
    df['Reports 1 day ago'] = shift_crimes(df,1)
    df['Reports 2 days ago'] = shift_crimes(df,2)
    df['Reports 3 days ago'] = shift_crimes(df,3)
    df['Reports 4 days ago'] = shift_crimes(df,4)
    df['Reports 5 days ago'] = shift_crimes(df,5)
    df['Reports 6 days ago'] = shift_crimes(df,6)
    df['Reports 7 days ago'] = shift_crimes(df,7)
    df['Reports 14 days ago'] = shift_crimes(df,14)
    df['Reports 30 days ago'] = shift_crimes(df,30)
    df['Reports 365 days ago'] = shift_crimes(df,365)
    df['Last 7 days reports'] = (df['Reports 1 day ago'] + df['Reports 2 days ago'] + df['Reports 3 days ago']
                         + df['Reports 4 days ago'] + df['Reports 5 days ago']
                         + df['Reports 6 days ago'] + df['Reports 7 days ago'])
    df['Last 14 days reports'] = (df['Last 7 days reports'] + get_last_14_days(df))
    df['Last 28 days reports'] = (df['Last 14 days reports'] + get_last_28_days(df))
    df = df.dropna()
    return df

In [25]:
# reset dataframe indexes to allow for iteration
def reset_indexes(df):
    df.reset_index(drop=True, inplace=True)
    return df

In [26]:
# shift indcident data by a number of days in order to create historic incident columns
def shift_crimes(df,num_days):
    transitional_data = df.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Todays Reports']
    return transitional_data

<h1>Get business data<h1>

In [27]:
# open original business data file
def open_businesses():
    df = pd.read_csv(business_data_name)
    return df

In [28]:
# drop columns deemed unecessary from business data file
def drop_unnecessary_columns(df):
    df = df.drop(columns=['Analysis Neighborhoods','Current Supervisor Districts',
                          'Current Police Districts','SF Find Neighborhoods','Neighborhoods',
                          'LIC Code Description','LIC Code','Transient Occupancy Tax','Parking Tax',
                          'NAICS Code','Mail State','Mail City','Location Id','Business Account Number',
                          'Business Start Date','Business End Date','Ownership Name','Street Address',
                          'City','State','Source Zipcode','Mail Address','Mail Zipcode',
                          'Supervisor District','Business Corridor','NAICS Code Description','UniqueID'])
    return df

In [29]:
# drop incidents prior to uber legislation change
#adjusted crime dataset starts 01/07/2019, business closed one year prior to this are not required 
def drop_end_dates_before_uber_legislation_change(df):
    # give none closed businesses an arbitray future closure date
    df['Location End Date'] = df['Location End Date'].fillna(future_date)
    #convert all date strings to datetime
    df['Location End Date'] = pd.to_datetime(df['Location End Date'],dayfirst=True)
    df = df[~(df['Location End Date'].dt.date < one_year_before_start_date)]
    return df

In [30]:
# remove businesses without GPS coordinates and place other businesses inside a naeighbourhood polygon using GPS coordinates
def determine_neighborhood_by_geolocation(df, neighborhood_polygons,neighborhoods):
    #replace blank gps coords with arbitrary point a significant distance from san francisco, in the ocean
    df['Business Location'] = df['Business Location'].fillna('POINT (-120.0 30.00)')
    df['Business Location'] = df['Business Location'].str[7:-1]
    df[['Longitude','Latitude']] = df['Business Location'].str.split(" ", expand = True)
    df = convert_gps_to_neighborhood_businesses(df,neighborhood_polygons,neighborhoods)
    return df

In [31]:
# place businesses inside a naeighbourhood polygon using GPS coordinates
def convert_gps_to_neighborhood_businesses(df,neighborhood_polygons,neighborhoods):
    df['Neighborhood'] = np.nan
    num_records = len(df.index)
    for business_num in range(0,num_records):
        df.iloc[business_num,7] = ''
        if df.iloc[business_num,4] == '-120.0 30.00':
            for neighborhood in neighborhoods:
                if df.iloc[business_num,3] == neighborhood:
                    df.iloc[business_num,7] = neighborhood
            if df.iloc[business_num,7] == '':
                df.iloc[business_num,7] = 'no gps'     
        else:
            shortest_distance = 999999999
            closest_neighborhood = ''
            latitude = float(df.iloc[business_num,6])
            longitude = float(df.iloc[business_num,5])
            point = Point(latitude,longitude)
            for poly_tuple in neighborhood_polygons:
                if poly_tuple[0].contains(point) or poly_tuple[0].touches(point):
                    df.iloc[business_num,7] = poly_tuple[1]
                    break
                else:
                    distance_to_neighborhood = point.distance(poly_tuple[0])
                    if distance_to_neighborhood < shortest_distance:
                        shortest_distance = distance_to_neighborhood
                        closest_neighborhood = poly_tuple[1]
            if df.iloc[business_num,7] == '':
                #if less than half a mile from a neighborhood polygon and sufficiently north to be san francisco
                if shortest_distance < 0.018 and latitude > 37.709126:
                    df.iloc[business_num,7] = closest_neighborhood
    df.reset_index(drop=True, inplace=True)
    df = df[df.Neighborhood != '']
    df = df[df.Neighborhood != 'no gps']
    return df

In [32]:
# remove additional uneccessary columns
def cleanup_columns(df):
    df = df.drop(columns=['Neighborhoods - Analysis Boundaries',
                          'Business Location',
                          'Longitude','Latitude'])
    return df

In [33]:
# determine number of business closures per day per neighbourhood
def get_closures_per_day_per_neighborhood(closures):
    closures['Closures'] = 1
    closures['Location End Date'] = pd.to_datetime(closures['Location End Date'],dayfirst=True)
    closures = closures[closures['Location End Date'].dt.year < future_year]
    closures = closures.groupby(['Location End Date','Neighborhood']).count()
    closures = closures.drop(columns=[ 'DBA Name','Location Start Date'])
    closures = closures.reset_index()
    return closures

In [34]:
# add rows for neighbourhoods which had zero business closures in a given day
def add_zero_closure_neighborhoods(closures, neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(closures.index)
    current_date = closures.iloc[0,0]
    zero_closure_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        while closures.iloc[record,0] != current_date:
            for neighborhood in zero_closure_neighborhoods:
                new_row = {'Location End Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Closures' : 0}
                new_rows = new_rows.append(new_row,ignore_index=True)
            current_date = current_date + timedelta(days=1)
        if closures.iloc[record,0] == current_date:
            zero_closure_neighborhoods.remove(closures.iloc[record,1])
            if record != num_records-1:
                if closures.iloc[record+1,0] != current_date:
                    for neighborhood in zero_closure_neighborhoods:
                        new_row = {'Location End Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Closures' : 0}
                        new_rows = new_rows.append(new_row,ignore_index=True)
                    zero_closure_neighborhoods = copy.deepcopy(neighborhoods)
                    current_date = current_date + timedelta(days=1)
        if current_date == date_after_end_date:
            break
    for neighborhood in zero_closure_neighborhoods:
        new_row = {'Location End Date' : current_date,
                    'Neighborhood' : neighborhood,
                    'Closures' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [closures,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Location End Date','Neighborhood'])
    return new_record

In [35]:
# shift closures by a number of days in order to create historic data
def shift_closures(closures,num_days):
    transitional_data = closures.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Closures']
    return transitional_data

In [36]:
# generate the last 14 days business closures
def get_last_14_days_closures(closures):
    transitional_data = closures.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days closures']
    return transitional_data

In [37]:
# generate the last 28 days business closures
def get_last_28_days_closures(closures):
    transitional_data = closures.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days closures']
    return transitional_data

In [38]:
# generate historic business closure data
def generate_new_closure_date_data(closures):
    closures['Closures 1 day ago'] = shift_closures(closures,1)
    closures['Closures 2 days ago'] = shift_closures(closures,2)
    closures['Closures 3 days ago'] = shift_closures(closures,3)
    closures['Closures 4 days ago'] = shift_closures(closures,4)
    closures['Closures 5 days ago'] = shift_closures(closures,5)
    closures['Closures 6 days ago'] = shift_closures(closures,6)
    closures['Closures 7 days ago'] = shift_closures(closures,7)
    closures['Closures 14 days ago'] = shift_closures(closures,14)
    closures['Closures 30 days ago'] = shift_closures(closures,30)
    closures['Closures 365 days ago'] = shift_closures(closures,365)
    closures['Last 7 days closures'] = (closures['Closures 1 day ago']
                                        + closures['Closures 2 days ago']
                                        + closures['Closures 3 days ago']
                                        + closures['Closures 4 days ago']
                                        + closures['Closures 5 days ago']
                                        + closures['Closures 6 days ago']
                                        + closures['Closures 7 days ago'])
    closures['Last 14 days closures'] = (closures['Last 7 days closures']
                                         + get_last_14_days_closures(closures))
    closures['Last 28 days closures'] = (closures['Last 14 days closures']
                                         + get_last_28_days_closures(closures))
    closures = closures.dropna()
    closures = closures[closures['Location End Date'] >= business_start_date]
    closures = closures[closures['Location End Date'] <= business_end_date]
    return closures

In [39]:
# determine the number of business openings per neighbourhood
def get_openings_per_day_per_neighborhood(openings):
    openings['Openings'] = 1
    openings['Location Start Date'] = pd.to_datetime(openings['Location Start Date'],dayfirst=True)
    openings = openings[openings['Location Start Date'].dt.year >= start_year-1]
    openings = openings.groupby(['Location Start Date','Neighborhood']).count()
    openings = openings.drop(columns=[ 'DBA Name','Location End Date'])
    openings = openings.reset_index()
    return openings

In [40]:
# add rows for any neighbourhood with zero business openings on a given day
def add_zero_opening_neighborhoods(openings, neighborhoods):
    new_rows = pd.DataFrame()
    num_records = len(openings.index)
    current_date = openings.iloc[0,0]
    zero_opening_neighborhoods = copy.deepcopy(neighborhoods)
    for record in range(0,num_records):
        while openings.iloc[record,0] != current_date:
            for neighborhood in zero_opening_neighborhoods:
                new_row = {'Location Start Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Openings' : 0}
                new_rows = new_rows.append(new_row,ignore_index=True)
            current_date = current_date + timedelta(days=1)
        if openings.iloc[record,0] == current_date:
            zero_opening_neighborhoods.remove(openings.iloc[record,1])
            if record != num_records-1:
                if openings.iloc[record+1,0] != current_date:
                    for neighborhood in zero_opening_neighborhoods:
                        new_row = {'Location Start Date' : current_date,
                                   'Neighborhood' : neighborhood,
                                   'Openings' : 0}
                        new_rows = new_rows.append(new_row,ignore_index=True)
                    zero_opening_neighborhoods = copy.deepcopy(neighborhoods)
                    current_date = current_date + timedelta(days=1)
        if current_date == date_after_end_date:
            break
    for neighborhood in zero_opening_neighborhoods:
        new_row = {'Location Start Date' : current_date,
                    'Neighborhood' : neighborhood,
                    'Openings' : 0}
        new_rows = new_rows.append(new_row,ignore_index=True)
    frames = [openings,new_rows]
    new_record = pd.concat(frames)
    new_record = new_record.sort_values(['Location Start Date','Neighborhood'])
    return new_record

In [41]:
# shift business openings by a number of days to create hitoric data
def shift_openings(openings,num_days):
    transitional_data = openings.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Openings']
    return transitional_data

In [42]:
# generate the last 14 days business openings data
def get_last_14_days_openings(openings):
    transitional_data = openings.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days openings']
    return transitional_data

In [43]:
# generate the last 28 days business openings data
def get_last_28_days_openings(openings):
    transitional_data = openings.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days openings']
    return transitional_data

In [44]:
# generate historic business openings data
def generate_new_opening_date_data(openings):
    openings['Openings 1 day ago'] = shift_openings(openings,1)
    openings['Openings 2 days ago'] = shift_openings(openings,2)
    openings['Openings 3 days ago'] = shift_openings(openings,3)
    openings['Openings 4 days ago'] = shift_openings(openings,4)
    openings['Openings 5 days ago'] = shift_openings(openings,5)
    openings['Openings 6 days ago'] = shift_openings(openings,6)
    openings['Openings 7 days ago'] = shift_openings(openings,7)
    openings['Openings 14 days ago'] = shift_openings(openings,14)
    openings['Openings 30 days ago'] = shift_openings(openings,30)
    openings['Openings 365 days ago'] = shift_openings(openings,365)
    openings['Last 7 days openings'] = (openings['Openings 1 day ago']
                                        + openings['Openings 2 days ago']
                                        + openings['Openings 3 days ago']
                                        + openings['Openings 4 days ago']
                                        + openings['Openings 5 days ago']
                                        + openings['Openings 6 days ago']
                                        + openings['Openings 7 days ago'])
    openings['Last 14 days openings'] = (openings['Last 7 days openings']
                                         + get_last_14_days_openings(openings))
    openings['Last 28 days openings'] = (openings['Last 14 days openings']
                                         + get_last_28_days_openings(openings))
    openings = openings.dropna()
    openings = openings[openings['Location Start Date'] >= business_start_date]
    openings = openings[openings['Location Start Date'] <= business_end_date]
    return openings

In [45]:
# generate data placing each open business in each date in which it was trading
def generate_dates_for_each_business(df):
    df['Date'] = np.nan
    df['Date']= pd.to_datetime(df['Date'],dayfirst=True)
    df['Location Start Date']= pd.to_datetime(df['Location Start Date'],dayfirst=True)
    df['Location End Date']= pd.to_datetime(df['Location End Date'],dayfirst=True)
    new_df = pd.DataFrame()
    final_df = pd.DataFrame()
    num_records = len(df.index)
    current_date = one_year_before_start_date #one year prior to beginning of SFPD data
    #end_date = end_date #final date of SFPD data
    delta = timedelta(days = 1)
    while current_date <= end_date:
        num_records = len(df.index)
        new_df = (df.loc[(pd.to_datetime(df['Location Start Date'],dayfirst=True).dt.date <= current_date) 
                         & (current_date < pd.to_datetime(df['Location End Date'],dayfirst=True).dt.date)])
        new_df['Date'] = current_date
        final_df = pd.concat((new_df,final_df))
        current_date += delta
    return final_df

In [46]:
# group businesses by date and neighbourhood, giving open businesses per day per neighbourhood
def group_neighbourhoods(business_df):
    business_df['Number of businesses'] = 1
    business_df = business_df.groupby(['Date','Neighborhood'], as_index=False).count()
    business_df = business_df.drop(columns=[ 'DBA Name','Location Start Date',
                           'Location End Date'])
    return business_df

In [47]:
# shift open businesses by a number of days to create historic data
def shift_days(df,num_days):
    transitional_data = df.shift(periods=(num_neighbourhoods*num_days))
    transitional_data = transitional_data['Number of businesses']
    return transitional_data

In [48]:
# determine last 14 days open businesses
def get_last_14_days_bus(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*7))
    transitional_data = transitional_data['Last 7 days']
    return transitional_data

In [49]:
# determine last 28 days open businesses
def get_last_28_days_bus(df):
    transitional_data = df.shift(periods=(num_neighbourhoods*14))
    transitional_data = transitional_data['Last 14 days']
    return transitional_data

In [50]:
# generate historic open business data
def generate_new_date_data_business(df):
    df['Businesses 1 day ago'] = shift_days(df,1)
    df['Businesses 2 days ago'] = shift_days(df,2)
    df['Businesses 3 days ago'] = shift_days(df,3)
    df['Businesses 4 days ago'] = shift_days(df,4)
    df['Businesses 5 days ago'] = shift_days(df,5)
    df['Businesses 6 days ago'] = shift_days(df,6)
    df['Businesses 7 days ago'] = shift_days(df,7)
    df['Businesses 14 days ago'] = shift_days(df,14)
    df['Businesses 30 days ago'] = shift_days(df,30)
    df['Businesses 365 days ago'] = shift_days(df,365)
    df = df.dropna()
    return df

In [51]:
# add rows for neighbourhoods which had zero open businesses on any given day
def add_missing_neighborhoods(business_df):
    new_rows = pd.DataFrame()
    num_records = len(business_df.index)
    current_date = one_year_before_start_date
    zero_business_neighborhoods = copy.deepcopy(neighborhoods)
    while current_date <= end_date:
        data = business_df.loc[business_df['Date']==current_date]
        for neighborhood in neighborhoods:
            if neighborhood not in data['Neighborhood'].tolist():
                new_row = {'Date' : current_date,
                               'Neighborhood' : neighborhood,
                               'Number of businesses' : 0}
                new_rows = new_rows.append(new_row,ignore_index=True)
        current_date += timedelta(days=1)
    frames = [business_df,new_rows]
    new_record = pd.concat(frames)
    new_record.reset_index(drop=True, inplace=True)
    new_record = new_record.sort_values(['Date','Neighborhood'])
    return new_record

<h1>Merge dataframes<h1>

In [52]:
# merge the four sub-datasets into one, renaming columns where necessary
def finalise_data(sfpd_data,businesses,openings,closures):
    sfpd_data.rename(columns={"1 day ago": "Crime 1 day ago",
                              "2 days ago": "Crime 2 days ago",
                              "3 days ago": "Crime 3 days ago",
                              "4 days ago": "Crime 4 days ago",
                              "5 days ago": "Crime 5 days ago",
                              "6 days ago": "Crime 6 days ago",
                              "7 days ago": "Crime 7 days ago",
                              "14 days ago": "Crime 14 days ago",
                              "30 days ago": "Crime 30 days ago",
                              "365 days ago": "Crime 365 days ago",
                              "Last 7 days": "Last 7 days crime",
                              "Last 14 days": "Last 14 days crime",
                              "Last 28 days": "Last 28 days crime"})
    sfpd_data['Number of businesses'] = businesses['Number of businesses']
    sfpd_data['Businesses 1 day ago'] = businesses['Businesses 1 day ago']
    sfpd_data['Businesses 2 days ago'] = businesses['Businesses 2 days ago']
    sfpd_data['Businesses 3 days ago'] = businesses['Businesses 3 days ago']
    sfpd_data['Businesses 4 days ago'] = businesses['Businesses 4 days ago']
    sfpd_data['Businesses 5 days ago'] = businesses['Businesses 5 days ago']
    sfpd_data['Businesses 6 days ago'] = businesses['Businesses 6 days ago']
    sfpd_data['Businesses 7 days ago'] = businesses['Businesses 7 days ago']
    sfpd_data['Businesses 14 days ago'] = businesses['Businesses 14 days ago']
    sfpd_data['Businesses 30 days ago'] = businesses['Businesses 30 days ago']
    sfpd_data['Businesses 365 days ago'] = businesses['Businesses 365 days ago']
    sfpd_data['Number of closures'] = closures['Closures']
    sfpd_data['Closures 1 day ago'] = closures['Closures 1 day ago']
    sfpd_data['Closures 2 days ago'] = closures['Closures 2 days ago']
    sfpd_data['Closures 3 days ago'] = closures['Closures 3 days ago']
    sfpd_data['Closures 4 days ago'] = closures['Closures 4 days ago']
    sfpd_data['Closures 5 days ago'] = closures['Closures 5 days ago']
    sfpd_data['Closures 6 days ago'] = closures['Closures 6 days ago']
    sfpd_data['Closures 7 days ago'] = closures['Closures 7 days ago']
    sfpd_data['Closures 14 days ago'] = closures['Closures 14 days ago']
    sfpd_data['Closures 30 days ago'] = closures['Closures 30 days ago']
    sfpd_data['Closures 365 days ago'] = closures['Closures 365 days ago']
    sfpd_data['Last 7 days closures'] = closures['Last 7 days closures']
    sfpd_data['Last 14 days closures'] = closures['Last 14 days closures']
    sfpd_data['Last 28 days closures'] = closures['Last 28 days closures']
    sfpd_data['Number of openings'] = openings['Openings']
    sfpd_data['Openings 1 day ago'] = openings['Openings 1 day ago']
    sfpd_data['Openings 2 days ago'] = openings['Openings 2 days ago']
    sfpd_data['Openings 3 days ago'] = openings['Openings 3 days ago']
    sfpd_data['Openings 4 days ago'] = openings['Openings 4 days ago']
    sfpd_data['Openings 5 days ago'] = openings['Openings 5 days ago']
    sfpd_data['Openings 6 days ago'] = openings['Openings 6 days ago']
    sfpd_data['Openings 7 days ago'] = openings['Openings 7 days ago']
    sfpd_data['Openings 14 days ago'] = openings['Openings 14 days ago']
    sfpd_data['Openings 30 days ago'] = openings['Openings 30 days ago']
    sfpd_data['Openings 365 days ago'] = openings['Openings 365 days ago']
    sfpd_data['Last 7 days openings'] = openings['Last 7 days openings']
    sfpd_data['Last 14 days openings'] = openings['Last 14 days openings']
    sfpd_data['Last 28 days openings'] = openings['Last 28 days openings']
    sfpd_data = sfpd_data.drop(columns=['Incident Year'])
    sfpd_data = sfpd_data.rename(columns={'Incident Date': 'Date'})
    return sfpd_data

In [53]:
# convert the day of week into one hot encoded columns
def convert_day_of_week(df):
    dummies = pd.get_dummies(df['Incident Day of Week'])
    merged = pd.concat([df,dummies],axis='columns')
    try:
        final = merged.drop(['Incident Day of Week','Monday'],axis='columns')
        return final
    except:
        final = merged.drop(['Incident Day of Week'],axis='columns')
        return final

In [54]:
# convert neighbourhoods into one hot encoded columns
def convert_neighborhood(df):
    dummies = pd.get_dummies(df['Neighborhood'])
    merged = pd.concat([df,dummies],axis='columns')
    final = merged.drop(['Ashbury Heights'],axis='columns')
    return final

In [55]:
# normalise all data with min/max normalisation
def normalise_data(df):
    y_data = df[['Todays Reports']]
    dates = df[['Date']]
    neighborhoods = df[['Neighborhood']]
    x_data = df.drop(columns=['Todays Reports','Date','Neighborhood'])
    x_data = x_data.apply(lambda x: (x - x.min(axis = 0)) / (x.max(axis = 0) - x.min(axis = 0)))
    x_data = x_data.fillna(1)
    x_data['Todays Reports'] = y_data['Todays Reports']
    x_data['Date'] = dates['Date']
    x_data['Neighborhood'] = neighborhoods['Neighborhood']
    return x_data

In [66]:
# main method to be called to generate a dataset, if is_test: additional columns retained to facilitate testing
def generate_dataset(is_test):
    # Incident data generation
    sfpd_df = open_sfpd_data()
    sfpd_df = remove_uncategorised_incidents(sfpd_df)
    sfpd_df = drop_unnecessary_incident_columns(sfpd_df)
    if not is_test:
        sfpd_df = sfpd_df.drop(columns=['Police District'])
    sfpd_df = remove_rows_with_missing_values(sfpd_df)
    sfpd_df = remove_rows_of_non_crime(sfpd_df)
    sfpd_df = remove_crime_prior_to_start_date(sfpd_df)
    if not is_test:
        sfpd_df = convert_gps_to_neighborhood(sfpd_df,neighborhood_polygons)
    else:
        sfpd_df = sfpd_df.rename(columns={'Police District':'Neighborhood'})
    sfpd_df = remove_gps_coordinates(sfpd_df)
    sfpd_df = get_reports_per_day_per_neighborhood(sfpd_df)
    sfpd_df = check_for_missing_neighborhoods(sfpd_df,neighborhoods)
    sfpd_df = reset_indexes(sfpd_df)
    sfpd_df = generate_new_date_data(sfpd_df)
    sfpd_df = reset_indexes(sfpd_df)
    business_df = open_businesses()
    business_df = drop_unnecessary_columns(business_df)
    business_df = drop_end_dates_before_uber_legislation_change(business_df)
    if not is_test:
        business_df = determine_neighborhood_by_geolocation(business_df, neighborhood_polygons,neighborhoods)
        business_df = cleanup_columns(business_df)
    else:
        business_df = business_df.rename(columns={'Neighborhoods - Analysis Boundaries':'Neighborhood'})
        business_df = business_df.drop(columns=['Business Location'])
    closures = business_df.copy(deep=True)
    closures = get_closures_per_day_per_neighborhood(closures)
    closures = add_zero_closure_neighborhoods(closures,neighborhoods)
    closures = generate_new_closure_date_data(closures)
    openings = business_df.copy(deep=True)
    openings = get_openings_per_day_per_neighborhood(openings)
    openings = add_zero_opening_neighborhoods(openings,neighborhoods)
    openings = generate_new_opening_date_data(openings)
    business_df = generate_dates_for_each_business(business_df)
    business_df = group_neighbourhoods(business_df)
    business_df = add_missing_neighborhoods(business_df)
    business_df = generate_new_date_data_business(business_df)
    business_df.reset_index(drop=True, inplace=False)
    # remove any trailing dates
    sfpd_df = sfpd_df[~(pd.to_datetime(sfpd_df['Incident Date'],dayfirst=True).dt.date > end_date)]
    openings = openings[~(pd.to_datetime(openings['Location Start Date'],dayfirst=True).dt.date > end_date)]
    closures = closures[~(pd.to_datetime(closures['Location End Date'],dayfirst=True).dt.date > end_date)]
    business_df.reset_index(drop=True, inplace=True)
    openings.reset_index(drop=True, inplace=True)
    closures.reset_index(drop=True, inplace=True)
    final_data = finalise_data(sfpd_df,business_df,openings,closures)
    final_data = convert_neighborhood(final_data)
    final_data = convert_day_of_week(final_data)
    final_data = normalise_data(final_data)
    final_data.to_csv(final_data_name, index = False)

In [569]:
generate_dataset(False)

c:\users\chris\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-557-d9cd5d55ce56>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = current_date


<h3>Tests<h3>

In [20]:
df = pd.read_csv(final_data_name)
neighbourhoods_df = pd.read_csv(neighbourhood_data_name)
# intended values for dataset
days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
expected_columns = ['Reports 1 day ago','Reports 2 days ago','Reports 3 days ago','Reports 4 days ago','Reports 5 days ago',
                    'Reports 6 days ago','Reports 7 days ago','Reports 14 days ago','Reports 30 days ago',
                    'Reports 365 days ago','Last 7 days reports','Last 14 days reports','Last 28 days reports',
                    'Number of businesses','Businesses 1 day ago','Businesses 2 days ago','Businesses 3 days ago',
                    'Businesses 4 days ago','Businesses 5 days ago','Businesses 6 days ago','Businesses 7 days ago',
                    'Businesses 14 days ago','Businesses 30 days ago','Businesses 365 days ago','Number of closures',
                    'Closures 1 day ago','Closures 2 days ago','Closures 3 days ago','Closures 4 days ago',
                    'Closures 5 days ago','Closures 6 days ago','Closures 7 days ago','Closures 14 days ago',
                    'Closures 30 days ago','Closures 365 days ago','Last 7 days closures','Last 14 days closures',
                    'Last 28 days closures','Number of openings','Openings 1 day ago','Openings 2 days ago',
                    'Openings 3 days ago','Openings 4 days ago','Openings 5 days ago','Openings 6 days ago',
                    'Openings 7 days ago','Openings 14 days ago','Openings 30 days ago','Openings 365 days ago',
                    'Last 7 days openings','Last 14 days openings','Last 28 days openings','Alamo Square','Anza Vista',
                    'Apparel City','Aquatic Park / Ft. Mason','Balboa Terrace','Bayview','Bernal Heights','Bret Harte',
                    'Buena Vista','Candlestick Point SRA','Castro','Cathedral Hill','Cayuga','Central Waterfront','Chinatown',
                    'Civic Center','Clarendon Heights','Cole Valley','Corona Heights','Cow Hollow','Crocker Amazon',
                    'Diamond Heights','Dogpatch','Dolores Heights','Downtown / Union Square','Duboce Triangle','Eureka Valley',
                    'Excelsior','Fairmount','Financial District','Fishermans Wharf','Forest Hill','Forest Knolls','Glen Park',
                    'Golden Gate Heights','Golden Gate Park','Haight Ashbury','Hayes Valley','Holly Park','Hunters Point',
                    'India Basin','Ingleside','Ingleside Terraces','Inner Richmond','Inner Sunset','Japantown','Laguna Honda',
                    'Lake Street','Lakeshore','Laurel Heights / Jordan Park','Lincoln Park / Ft. Miley','Little Hollywood',
                    'Lone Mountain','Lower Haight','Lower Nob Hill','Lower Pacific Heights','Marina','McLaren Park',
                    'Merced Heights','Merced Manor','Midtown Terrace','Mint Hill','Miraloma Park','Mission','Mission Bay',
                    'Mission Dolores','Mission Terrace','Monterey Heights','Mt. Davidson Manor','Nob Hill','Noe Valley',
                    'North Beach','Northern Waterfront','Oceanview','Outer Mission','Outer Richmond','Outer Sunset',
                    'Pacific Heights','Panhandle','Parkmerced','Parkside','Parnassus Heights','Peralta Heights',
                    'Polk Gulch','Portola','Potrero Hill','Presidio Heights','Presidio National Park','Presidio Terrace',
                    'Produce Market','Rincon Hill','Russian Hill','Seacliff','Sherwood Forest','Showplace Square',
                    'Silver Terrace','South Beach','South of Market','St. Francis Wood','St. Marys Park','Stonestown',
                    'Sunnydale','Sunnyside','Sutro Heights','Telegraph Hill','Tenderloin','Treasure Island','Union Street',
                    'University Mound','Upper Market','Visitacion Valley','West Portal','Western Addition',
                    'Westwood Highlands','Westwood Park','Yerba Buena Island','Friday','Saturday','Sunday','Thursday',
                    'Tuesday','Wednesday','Todays Reports']

In [22]:
# assert values for original dataset used for model training
sfpd_df = pd.read_csv('SFPD_preprocessed.csv')
expected_num_records = 72*117
assert expected_num_records == len(df), "Expected " + str(expected_num_records) + " records but got " + str(len(df))
print("Number of records in dataset is as expected.")
actual_columns = df.columns
for i in range(len(expected_columns)):
    assert expected_columns[i] == actual_columns[i], "Expected column " + expected_columns[i] + " but got " + actual_columns[i]
print("Column names of dataset are as expected.")
for record_num in range(0,len(df.index)):
    j = 0
    while j < len(expected_columns) - 1:
        assert df[expected_columns[j]].iloc[record_num] >= 0 and df[expected_columns[j]].iloc[record_num] <= 1, "Value for " + expected_columns[j] + " is less than 0"
        j += 1
    assert df['Todays Reports'].iloc[record_num] >= 0 , "Value for Todays Reports is less than 0"
print("All values in " + final_data_name + " dataset are as expected.")
print("\nAll tests completed successfully.")

Number of records in dataset is as expected.
Column names of dataset are as expected.
All values in new_final_data.csv dataset are as expected.

All tests completed successfully.


<h3>Define values for fake dataset tests<h3>

In [23]:
neighbourhood_data_name = "SF_neighborhoods.csv"
sfpd_data_name = "mock_SFPD_data.csv"
business_data_name = "mock_business_data.csv"
final_data_name = "mock_final_data.csv"
start_day = 5
start_month = 1
start_year = 2021
end_day = 11
end_month = 1
end_year = 2021

In [24]:
start_date_string = str(start_year)+'-'+str(start_month)+'-'+str(start_day)
start_date = date(start_year,start_month,start_day)
one_year_before_start_date = start_date - timedelta(days=365)
end_date = date (end_year,end_month,end_day)
date_after_end_date = end_date + timedelta(days=1)
future_year_increment = 50
future_date = str(end_year+future_year_increment)+'-'+str(end_month)+'-'+str(end_day)
future_year = end_year + future_year_increment
business_start_date = str(start_month)+'-'+str(start_day)+'-'+str(start_year)
business_end_date = str(end_month)+'-'+str(end_day)+'-'+str(end_year)

<h3>Generate fake dataset<h3>

In [574]:
generate_dataset(True)

<ipython-input-557-d9cd5d55ce56>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = current_date


<h3>Test fake dataset values<h3>

In [25]:
df = pd.read_csv(final_data_name)
manual_set = pd.read_csv("final_data_manually_created_for_testing.csv")
expected_num_records = 117*7
assert expected_num_records == len(df), "Expected " + str(expected_num_records) + " records but got " + str(len(df))
print("Number of records in dataset is as expected.")
actual_columns = df.columns
for i in range(len(expected_columns)):
    assert expected_columns[i] == actual_columns[i], "Expected column " + expected_columns[i] + " but got " + actual_columns[i]
print("Column names of dataset are as expected.")
for record_num in range(0,len(df.index)):
    j = 0
    while j <= len(expected_columns) - 1:
        actual = df[expected_columns[j]].iloc[record_num]
        expected = manual_set[expected_columns[j]].iloc[record_num]
        assert math.isclose(actual,expected, rel_tol=0.00000001), "Value on row " + str(record_num) + " for " + str(expected_columns[j]) + " column is " + str(actual) + " but " + str(expected) + " expected"
        j += 1
    assert df['Todays Reports'].iloc[record_num] >= 0 , "Value for Todays Reports is less than 0"
print("All values in dataset are as expected.")
print("\nAll tests completed successfully.")

Number of records in dataset is as expected.
Column names of dataset are as expected.
All values in dataset are as expected.

All tests completed successfully.


<h3>Generate original dataset<h3>

In [26]:
neighbourhood_data_name = "SF_neighborhoods.csv"
sfpd_data_name = "SFPD_original.csv"
business_data_name = "SF_registered_business_locations_2.csv" #Unique ID field added as was added to online dataset post download
final_data_name = "replica_finalised_data.csv"
start_day = 1
start_month = 7
start_year = 2019
end_day = 11
end_month = 11
end_year = 2020

In [27]:
start_date_string = str(start_year)+'-'+str(start_month)+'-'+str(start_day)
start_date = date(start_year,start_month,start_day)
one_year_before_start_date = start_date - timedelta(days=365)
end_date = date (end_year,end_month,end_day)
date_after_end_date = end_date + timedelta(days=1)
future_year_increment = 50
future_date = str(end_year+future_year_increment)+'-'+str(end_month)+'-'+str(end_day)
future_year = end_year + future_year_increment
business_start_date = str(start_month)+'-'+str(start_day)+'-'+str(start_year)
business_end_date = str(end_month)+'-'+str(end_day)+'-'+str(end_year)

In [69]:
generate_dataset(False)

c:\users\chris\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-45-d9cd5d55ce56>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Date'] = current_date


In [28]:
days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
expected_columns = ['Reports 1 day ago','Reports 2 days ago','Reports 3 days ago','Reports 4 days ago','Reports 5 days ago',
                    'Reports 6 days ago','Reports 7 days ago','Reports 14 days ago','Reports 30 days ago',
                    'Reports 365 days ago','Last 7 days reports','Last 14 days reports','Last 28 days reports',
                    'Number of businesses','Businesses 1 day ago','Businesses 2 days ago','Businesses 3 days ago',
                    'Businesses 4 days ago','Businesses 5 days ago','Businesses 6 days ago','Businesses 7 days ago',
                    'Businesses 14 days ago','Businesses 30 days ago','Businesses 365 days ago','Number of closures',
                    'Closures 1 day ago','Closures 2 days ago','Closures 3 days ago','Closures 4 days ago',
                    'Closures 5 days ago','Closures 6 days ago','Closures 7 days ago','Closures 14 days ago',
                    'Closures 30 days ago','Closures 365 days ago','Last 7 days closures','Last 14 days closures',
                    'Last 28 days closures','Number of openings','Openings 1 day ago','Openings 2 days ago',
                    'Openings 3 days ago','Openings 4 days ago','Openings 5 days ago','Openings 6 days ago',
                    'Openings 7 days ago','Openings 14 days ago','Openings 30 days ago','Openings 365 days ago',
                    'Last 7 days openings','Last 14 days openings','Last 28 days openings','Alamo Square','Anza Vista',
                    'Apparel City','Aquatic Park / Ft. Mason','Balboa Terrace','Bayview','Bernal Heights','Bret Harte',
                    'Buena Vista','Candlestick Point SRA','Castro','Cathedral Hill','Cayuga','Central Waterfront','Chinatown',
                    'Civic Center','Clarendon Heights','Cole Valley','Corona Heights','Cow Hollow','Crocker Amazon',
                    'Diamond Heights','Dogpatch','Dolores Heights','Downtown / Union Square','Duboce Triangle','Eureka Valley',
                    'Excelsior','Fairmount','Financial District','Fishermans Wharf','Forest Hill','Forest Knolls','Glen Park',
                    'Golden Gate Heights','Golden Gate Park','Haight Ashbury','Hayes Valley','Holly Park','Hunters Point',
                    'India Basin','Ingleside','Ingleside Terraces','Inner Richmond','Inner Sunset','Japantown','Laguna Honda',
                    'Lake Street','Lakeshore','Laurel Heights / Jordan Park','Lincoln Park / Ft. Miley','Little Hollywood',
                    'Lone Mountain','Lower Haight','Lower Nob Hill','Lower Pacific Heights','Marina','McLaren Park',
                    'Merced Heights','Merced Manor','Midtown Terrace','Mint Hill','Miraloma Park','Mission','Mission Bay',
                    'Mission Dolores','Mission Terrace','Monterey Heights','Mt. Davidson Manor','Nob Hill','Noe Valley',
                    'North Beach','Northern Waterfront','Oceanview','Outer Mission','Outer Richmond','Outer Sunset',
                    'Pacific Heights','Panhandle','Parkmerced','Parkside','Parnassus Heights','Peralta Heights',
                    'Polk Gulch','Portola','Potrero Hill','Presidio Heights','Presidio National Park','Presidio Terrace',
                    'Produce Market','Rincon Hill','Russian Hill','Seacliff','Sherwood Forest','Showplace Square',
                    'Silver Terrace','South Beach','South of Market','St. Francis Wood','St. Marys Park','Stonestown',
                    'Sunnydale','Sunnyside','Sutro Heights','Telegraph Hill','Tenderloin','Treasure Island','Union Street',
                    'University Mound','Upper Market','Visitacion Valley','West Portal','Western Addition',
                    'Westwood Highlands','Westwood Park','Yerba Buena Island','Friday','Saturday','Sunday','Thursday',
                    'Tuesday','Wednesday','Todays Reports']

<h3> Calculate data inconsistencies <h3>

In [29]:
def get_inconsistencies(tol):
    df = pd.read_csv(final_data_name)
    original_set = pd.read_csv("finalised_data.csv")
    expected_num_records = 117*500
    fields_with_inconsistencies = {}
    assert expected_num_records == len(df), "Expected " + str(expected_num_records) + " records but got " + str(len(df))
    print("Number of records in dataset is as expected.")
    actual_columns = df.columns
    for i in range(len(expected_columns)):
        assert expected_columns[i] == actual_columns[i], "Expected column " + expected_columns[i] + " but got " + actual_columns[i]
    print("Column names of dataset are as expected.")
    for record_num in range(0,len(df.index)):
        j = 0
        while j <= len(expected_columns) - 1:
            actual = df[expected_columns[j]].iloc[record_num]
            expected = original_set[expected_columns[j]].iloc[record_num]
            try:
                assert math.isclose(actual,expected, abs_tol=tol), "Value on row " + str(record_num) + " for " + str(expected_columns[j]) + " column is " + str(actual) + " but " + str(expected) + " expected"
            except:
                if expected_columns[j] not in fields_with_inconsistencies:
                    fields_with_inconsistencies[expected_columns[j]] = 1
                else:
                    current_val = fields_with_inconsistencies[expected_columns[j]]
                    current_val += 1
                    fields_with_inconsistencies[expected_columns[j]] = current_val
            j += 1
        assert df['Todays Reports'].iloc[record_num] >= 0 , "Value for Todays Reports is less than 0"
    if not fields_with_inconsistencies:
        print("All values in dataset are as expected within a " + str(tol) + "% tolerance")
    else:
        print("Some values in dataset vary beyond a " + str(tol) + "% tolerance.")
    print("\nAll tests completed successfully.")
    return fields_with_inconsistencies

In [30]:
one_pc_inconsistencies = get_inconsistencies(0.01)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.01% tolerance.

All tests completed successfully.


In [16]:
two_pc_inconsistencies = get_inconsistencies(0.02)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.02% tolerance.

All tests completed successfully.


In [17]:
three_pc_inconsistencies = get_inconsistencies(0.03)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.03% tolerance.

All tests completed successfully.


In [18]:
four_pc_inconsistencies = get_inconsistencies(0.04)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.04% tolerance.

All tests completed successfully.


In [19]:
five_pc_inconsistencies = get_inconsistencies(0.05)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.05% tolerance.

All tests completed successfully.


In [20]:
six_pc_inconsistencies = get_inconsistencies(0.06)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.06% tolerance.

All tests completed successfully.


In [21]:
seven_pc_inconsistencies = get_inconsistencies(0.07)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.07% tolerance.

All tests completed successfully.


In [22]:
eight_pc_inconsistencies = get_inconsistencies(0.08)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.08% tolerance.

All tests completed successfully.


In [23]:
nine_pc_inconsistencies = get_inconsistencies(0.09)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.09% tolerance.

All tests completed successfully.


In [24]:
ten_pc_inconsistencies = get_inconsistencies(0.1)

Number of records in dataset is as expected.
Column names of dataset are as expected.
Some values in dataset vary beyond a 0.1% tolerance.

All tests completed successfully.


In [26]:
inconsistencies = [one_pc_inconsistencies, two_pc_inconsistencies, three_pc_inconsistencies, four_pc_inconsistencies,
                  five_pc_inconsistencies, six_pc_inconsistencies, seven_pc_inconsistencies, eight_pc_inconsistencies,
                  nine_pc_inconsistencies, ten_pc_inconsistencies]

<h3> Get original features <h3>

In [29]:
def get_features(file_tag):
    features = []
    df = pd.read_csv("x_train_" + file_tag + ".csv")
    for col in df:
        features.append(col)
    return features

In [32]:
f_reg_features = get_features("f_regression")

In [33]:
chi2_features = get_features("chi2")

In [34]:
adaboost_features = get_features("adaboost")

In [35]:
equal_features = get_features("equal_crime_and_business")

In [36]:
all_bus_features = get_features("all_business")

In [68]:
num_records = len(pd.read_csv("finalised_data.csv"))

<h3> Calculate error tolerance <h3>

In [104]:
pd.options.display.float_format = "{:,.2f}".format

In [105]:
def calculate_error_tolerences(feature_name,features,inconsistencies):
    df = pd.DataFrame()
    df_pc = pd.DataFrame()
    df['Feature'] = features
    df_pc['Feature'] = features
    for i in range(0,10):
        vals = []
        vals_pc = []
        for feature in features:
            if feature in inconsistencies[i]:
                vals.append(inconsistencies[i][feature])
                vals_pc.append(inconsistencies[i][feature]/num_records*100)
            else:
                vals.append(0)
                vals_pc.append(0)
        df[str(i + 1) + '%'] = vals
        df_pc[str(i + 1) + '%'] = vals_pc
    return df, df_pc

In [106]:
df, df_pc = calculate_error_tolerences("f-regression",f_reg_features,inconsistencies)
print("Number of inconsistencies for f-regression dataset in tolerance range of 1% to 10%")
df

Number of inconsistencies for f-regression dataset in tolerance range of 1% to 10%


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Reports 1 day ago,0,0,0,0,0,0,0,0,0,0
1,Reports 2 days ago,0,0,0,0,0,0,0,0,0,0
2,Reports 3 days ago,0,0,0,0,0,0,0,0,0,0
3,Reports 4 days ago,0,0,0,0,0,0,0,0,0,0
4,Reports 5 days ago,0,0,0,0,0,0,0,0,0,0
5,Reports 6 days ago,0,0,0,0,0,0,0,0,0,0
6,Reports 7 days ago,0,0,0,0,0,0,0,0,0,0
7,Reports 14 days ago,0,0,0,0,0,0,0,0,0,0
8,Reports 30 days ago,0,0,0,0,0,0,0,0,0,0
9,Reports 365 days ago,0,0,0,0,0,0,0,0,0,0


In [107]:
print("Number of inconsistencies for f-regression dataset in tolerance range of 1% to 10% as a percentage of total records")
df_pc

Number of inconsistencies for f-regression dataset in tolerance range of 1% to 10% as a percentage of total records


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Reports 1 day ago,0,0,0,0,0,0,0,0,0,0
1,Reports 2 days ago,0,0,0,0,0,0,0,0,0,0
2,Reports 3 days ago,0,0,0,0,0,0,0,0,0,0
3,Reports 4 days ago,0,0,0,0,0,0,0,0,0,0
4,Reports 5 days ago,0,0,0,0,0,0,0,0,0,0
5,Reports 6 days ago,0,0,0,0,0,0,0,0,0,0
6,Reports 7 days ago,0,0,0,0,0,0,0,0,0,0
7,Reports 14 days ago,0,0,0,0,0,0,0,0,0,0
8,Reports 30 days ago,0,0,0,0,0,0,0,0,0,0
9,Reports 365 days ago,0,0,0,0,0,0,0,0,0,0


In [108]:
df, df_pc = calculate_error_tolerences("chi2",chi2_features,inconsistencies)
print("Number of inconsistencies for chi2 dataset in tolerance range of 1% to 10%")
df

Number of inconsistencies for chi2 dataset in tolerance range of 1% to 10%


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,South of Market,0,0,0,0,0,0,0,0,0,0
1,Mission,0,0,0,0,0,0,0,0,0,0
2,Tenderloin,0,0,0,0,0,0,0,0,0,0
3,Number of businesses,31,0,0,0,0,0,0,0,0,0
4,Downtown / Union Square,0,0,0,0,0,0,0,0,0,0
5,Civic Center,0,0,0,0,0,0,0,0,0,0
6,Reports 365 days ago,0,0,0,0,0,0,0,0,0,0
7,Reports 1 day ago,0,0,0,0,0,0,0,0,0,0
8,Reports 2 days ago,0,0,0,0,0,0,0,0,0,0
9,Reports 14 days ago,0,0,0,0,0,0,0,0,0,0


In [109]:
print("Number of inconsistencies for chi2 dataset in tolerance range of 1% to 10% as a percentage of total records")
df_pc

Number of inconsistencies for chi2 dataset in tolerance range of 1% to 10% as a percentage of total records


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,South of Market,0.00,0,0,0,0,0,0,0,0,0
1,Mission,0.00,0,0,0,0,0,0,0,0,0
2,Tenderloin,0.00,0,0,0,0,0,0,0,0,0
3,Number of businesses,0.05,0,0,0,0,0,0,0,0,0
4,Downtown / Union Square,0.00,0,0,0,0,0,0,0,0,0
5,Civic Center,0.00,0,0,0,0,0,0,0,0,0
6,Reports 365 days ago,0.00,0,0,0,0,0,0,0,0,0
7,Reports 1 day ago,0.00,0,0,0,0,0,0,0,0,0
8,Reports 2 days ago,0.00,0,0,0,0,0,0,0,0,0
9,Reports 14 days ago,0.00,0,0,0,0,0,0,0,0,0


In [110]:
df, df_pc = calculate_error_tolerences("adaboost",adaboost_features,inconsistencies)
print("Number of inconsistencies for adaboost dataset in tolerance range of 1% to 10%")
df

Number of inconsistencies for adaboost dataset in tolerance range of 1% to 10%


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Reports 365 days ago,0,0,0,0,0,0,0,0,0,0
1,Reports 1 day ago,0,0,0,0,0,0,0,0,0,0
2,Reports 14 days ago,0,0,0,0,0,0,0,0,0,0
3,Reports 3 days ago,0,0,0,0,0,0,0,0,0,0
4,Reports 2 days ago,0,0,0,0,0,0,0,0,0,0
5,Reports 7 days ago,0,0,0,0,0,0,0,0,0,0
6,Number of businesses,31,0,0,0,0,0,0,0,0,0
7,Reports 4 days ago,0,0,0,0,0,0,0,0,0,0
8,Reports 5 days ago,0,0,0,0,0,0,0,0,0,0
9,Closures 365 days ago,136,28,9,7,4,0,0,0,0,0


In [111]:
print("Number of inconsistencies for adaboost dataset in tolerance range of 1% to 10% as a percentage of total records")
df_pc

Number of inconsistencies for adaboost dataset in tolerance range of 1% to 10% as a percentage of total records


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Reports 365 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
1,Reports 1 day ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
2,Reports 14 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
3,Reports 3 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
4,Reports 2 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
5,Reports 7 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
6,Number of businesses,0.05,0.00,0.00,0.00,0.00,0,0,0,0,0
7,Reports 4 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
8,Reports 5 days ago,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0
9,Closures 365 days ago,0.23,0.05,0.02,0.01,0.01,0,0,0,0,0


In [112]:
df, df_pc = calculate_error_tolerences("equal crime and business",equal_features,inconsistencies)
print("Number of inconsistencies for equal crime and business dataset in tolerance range of 1% to 10%")
df

Number of inconsistencies for equal crime and business dataset in tolerance range of 1% to 10%


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Number of businesses,31,0,0,0,0,0,0,0,0,0
1,Last 28 days closures,1800,451,160,81,68,42,19,0,0,0
2,Last 7 days openings,4535,1787,949,503,343,222,160,118,96,87
3,Last 14 days closures,924,211,70,42,31,16,7,0,0,0
4,Last 7 days closures,417,97,31,18,9,3,0,0,0,0
5,Reports 1 day ago,0,0,0,0,0,0,0,0,0,0
6,Reports 2 days ago,0,0,0,0,0,0,0,0,0,0
7,Reports 4 days ago,0,0,0,0,0,0,0,0,0,0
8,Reports 30 days ago,0,0,0,0,0,0,0,0,0,0
9,Reports 7 days ago,0,0,0,0,0,0,0,0,0,0


In [113]:
print("Number of inconsistencies for equal crime and business dataset in tolerance range of 1% to 10% as a percentage of total records")
df_pc

Number of inconsistencies for equal crime and business dataset in tolerance range of 1% to 10% as a percentage of total records


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Number of businesses,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Last 28 days closures,3.08,0.77,0.27,0.14,0.12,0.07,0.03,0.00,0.00,0.00
2,Last 7 days openings,7.75,3.05,1.62,0.86,0.59,0.38,0.27,0.20,0.16,0.15
3,Last 14 days closures,1.58,0.36,0.12,0.07,0.05,0.03,0.01,0.00,0.00,0.00
4,Last 7 days closures,0.71,0.17,0.05,0.03,0.02,0.01,0.00,0.00,0.00,0.00
5,Reports 1 day ago,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Reports 2 days ago,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Reports 4 days ago,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,Reports 30 days ago,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Reports 7 days ago,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [114]:
df, df_pc = calculate_error_tolerences("all business",all_bus_features,inconsistencies)
print("Number of inconsistencies for all business dataset in tolerance range of 1% to 10%")
df

Number of inconsistencies for all business dataset in tolerance range of 1% to 10%


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Number of businesses,31,0,0,0,0,0,0,0,0,0
1,Last 28 days closures,1800,451,160,81,68,42,19,0,0,0
2,Last 7 days openings,4535,1787,949,503,343,222,160,118,96,87
3,Last 14 days closures,924,211,70,42,31,16,7,0,0,0
4,Last 7 days closures,417,97,31,18,9,3,0,0,0,0
5,Number of openings,668,211,95,64,42,27,19,16,9,7
6,Openings 4 days ago,665,209,95,64,42,27,19,16,9,7
7,Openings 1 day ago,668,211,95,64,42,27,19,16,9,7
8,Openings 7 days ago,665,209,95,64,42,27,19,16,9,7
9,Openings 2 days ago,668,211,95,64,42,27,19,16,9,7


In [115]:
print("Number of inconsistencies for all business dataset in tolerance range of 1% to 10% as a percentage of total records")
df_pc

Number of inconsistencies for all business dataset in tolerance range of 1% to 10% as a percentage of total records


,Feature,1%,2%,3%,4%,5%,6%,7%,8%,9%,10%
0,Number of businesses,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Last 28 days closures,3.08,0.77,0.27,0.14,0.12,0.07,0.03,0.00,0.00,0.00
2,Last 7 days openings,7.75,3.05,1.62,0.86,0.59,0.38,0.27,0.20,0.16,0.15
3,Last 14 days closures,1.58,0.36,0.12,0.07,0.05,0.03,0.01,0.00,0.00,0.00
4,Last 7 days closures,0.71,0.17,0.05,0.03,0.02,0.01,0.00,0.00,0.00,0.00
5,Number of openings,1.14,0.36,0.16,0.11,0.07,0.05,0.03,0.03,0.02,0.01
6,Openings 4 days ago,1.14,0.36,0.16,0.11,0.07,0.05,0.03,0.03,0.02,0.01
7,Openings 1 day ago,1.14,0.36,0.16,0.11,0.07,0.05,0.03,0.03,0.02,0.01
8,Openings 7 days ago,1.14,0.36,0.16,0.11,0.07,0.05,0.03,0.03,0.02,0.01
9,Openings 2 days ago,1.14,0.36,0.16,0.11,0.07,0.05,0.03,0.03,0.02,0.01


<h3> Significant errors in original 'equal crime and business' and 'all business'datasets.<h3> 

<h3>Model retraining required on these datasets.<h3>